In [1]:
#https://hackersandslackers.com/scraping-urls-with-beautifulsoup/
#https://codeburst.io/web-scraping-101-with-python-beautiful-soup-bb617be1f486

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup 
from nltk.corpus import stopwords

In [3]:
stop = stopwords.words('english')
# stop

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException

import pandas as pd
import numpy as np

In [5]:
page_link = 'http://mashable.com/2013/01/07/amazon-instant-video-browser/'
page_response = requests.get(page_link, timeout=5)
page_content = BeautifulSoup(page_response.content, "html.parser")
textContent = []
for i in range(0, len(page_content)):
    paragraphs = page_content.find_all("p")[i].text
    textContent.append(paragraphs)

In [6]:
textContent

["Having trouble finding something to watch on Amazon Instant Video? The retailer launched Monday an experimental browsing tool that lets users discover movies and TV shows based on their genre preferences or simply the mood they're in.",
 'Movies and shows are divided up into categories, some of which bear the names of genres (i.e., "Comedy" and "Mystery/Thriller"), and others which are labeled by mood, such as "Feel-Good" and "Exciting." Users can toggle between TV shows and movies, and apply filters to show only videos that are available for free viewing to Prime subscribers, or ones that bear G or PG ratings.',
 "It's pretty basic, but it sure beats the haphazard organization of Amazon's current Instant Video page, which mixes rows of new releases with bestsellers and personal recommendations.",
 "The move is the latest in a series of investment's Amazon is making in its streaming video platform. Last week, the company inked a licensing agreement with A+E Networks to bring past sea

In [7]:
paragraphs = page_content.find_all("p")[i].text

In [8]:
paragraphs

"The move is the latest in a series of investment's Amazon is making in its streaming video platform. Last week, the company inked a licensing agreement with A+E Networks to bring past seasons of shows from A&E, History, Lifetime and Bio to its streaming library — shows, it should be noted, that Netflix subscribers lost access to in the fall. Amazon claims its streaming Prime catalog now numbers more than 33,000, double what it was a year ago."

In [9]:
def getTitle(link):
    """Attempt to get a title."""
    title = ''
    if link.title.string is not None:
        title = link.title.string
    elif link.find("h1") is not None:
        title = link.find("h1")
    return title


def getDescription(link):
    """Attempt to get description."""
    description = ''
    if link.find("meta", property="og:description") is not None:
        description = link.find("meta", property="og:description").get('content')
    elif link.find("p") is not None:
        description = link.find("p").content
    return description


def getImage(link):
    """Attempt to get a preview image."""
    image = ''
    if link.find("meta", property="og:image") is not None:
        image = link.find("meta", property="og:image").get('content')
    elif link.find("img") is not None:
        image = link.find("img").get('href')
    return image


def getSiteName(link, url):
    """Attempt to get the site's base name."""
    sitename = ''
    if link.find("meta", property="og:site_name") is not None:
        sitename = link.find("meta", property="og:site_name").get('content')
    else:
        sitename = url.split('//')[1]
        name = sitename.split('/')[0]
        name = sitename.rsplit('.')[1]
        return name.capitalize()
    return sitename

In [10]:
def scrape(url):
    """Scrape scheduled link previews."""
    headers = requests.utils.default_headers()
    headers.update({
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
    })
    r = requests.get(url)
    raw_html = r.content
    soup = BeautifulSoup(raw_html, 'html.parser')
    links = soup.select('body p > a')
    previews = []
    for link in links:
        url = link.get('href')
        r2 = requests.get(url, headers=headers)
        link_html = r2.content
        embedded_link = BeautifulSoup(link_html, 'html.parser')
        link_preview_dict = {
            'title': getTitle(embedded_link),
            'description': getDescription(embedded_link),
            'image': getImage(embedded_link),
            'sitename': getSiteName(embedded_link, url),
            'url': url
            }
        previews.append(link_preview_dict)
        print(link_preview_dict)

In [11]:
scrape('http://mashable.com/2013/01/07/amazon-instant-video-browser/')

{'title': 'Amazon', 'description': 'Amazon, the working name of Amazon.com Inc., is an online retailer and cloud computing company founded by Jeff Bezos in 1994. Based in Seattle, Amazon.com is one of the largest internet retailers in the world. The website began as an online bookstore, and later began selling video downloads and streaming, MP3 downloads and streaming, audiobooks, software, video games, electronics, apparel, furniture, food, toys and jewelry. The company now produces its own line of consumer electronics including the Amazon Kindle, Amazon Fire Tablet, Amazon Fire TV, and Amazon Echo products. It is also the world’s largest provider of cloud services. In August 2017, Amazon acquired supermarket chain Whole Foods Market for $13.4 billion. In September 2017, it announced that it would build a second headquarters to compliment its Seattle campus, and is still reviewing applications from cities across the country that have applied. The company employs over 300,000 people wo

In [12]:
data = pd.read_csv("OnlineNewsPopularity.csv",sep='\s*,\s*',engine="python")
data

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.700000,-0.350000,-0.600000,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.700000,-0.118750,-0.125000,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.000000,-0.466667,-0.800000,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.800000,-0.369697,-0.600000,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.000000,-0.220192,-0.500000,-0.050000,0.454545,0.136364,0.045455,0.136364,505
5,http://mashable.com/2013/01/07/beewi-smart-toys/,731.0,10.0,370.0,0.559889,1.0,0.698198,2.0,2.0,0.0,...,0.136364,0.600000,-0.195000,-0.400000,-0.100000,0.642857,0.214286,0.142857,0.214286,855
6,http://mashable.com/2013/01/07/bodymedia-armba...,731.0,8.0,960.0,0.418163,1.0,0.549834,21.0,20.0,20.0,...,0.100000,1.000000,-0.224479,-0.500000,-0.050000,0.000000,0.000000,0.500000,0.000000,556
7,http://mashable.com/2013/01/07/canon-poweshot-n/,731.0,12.0,989.0,0.433574,1.0,0.572108,20.0,20.0,20.0,...,0.100000,1.000000,-0.242778,-0.500000,-0.050000,1.000000,0.500000,0.500000,0.500000,891
8,http://mashable.com/2013/01/07/car-of-the-futu...,731.0,11.0,97.0,0.670103,1.0,0.836735,2.0,0.0,0.0,...,0.400000,0.800000,-0.125000,-0.125000,-0.125000,0.125000,0.000000,0.375000,0.000000,3600
9,http://mashable.com/2013/01/07/chuck-hagel-web...,731.0,10.0,231.0,0.636364,1.0,0.797101,4.0,1.0,1.0,...,0.100000,0.500000,-0.238095,-0.500000,-0.100000,0.000000,0.000000,0.500000,0.000000,710


In [16]:
url=data["url"]
for each in url:
    print(each)

http://mashable.com/2013/01/07/amazon-instant-video-browser/
http://mashable.com/2013/01/07/ap-samsung-sponsored-tweets/
http://mashable.com/2013/01/07/apple-40-billion-app-downloads/
http://mashable.com/2013/01/07/astronaut-notre-dame-bcs/
http://mashable.com/2013/01/07/att-u-verse-apps/
http://mashable.com/2013/01/07/beewi-smart-toys/
http://mashable.com/2013/01/07/bodymedia-armbandgets-update/
http://mashable.com/2013/01/07/canon-poweshot-n/
http://mashable.com/2013/01/07/car-of-the-future-infographic/
http://mashable.com/2013/01/07/chuck-hagel-website/
http://mashable.com/2013/01/07/cosmic-events-doomsday/
http://mashable.com/2013/01/07/crayon-creatures/
http://mashable.com/2013/01/07/creature-cups/
http://mashable.com/2013/01/07/dad-jokes/
http://mashable.com/2013/01/07/downton-abbey-tumblrs/
http://mashable.com/2013/01/07/earth-size-planets-milky-way/
http://mashable.com/2013/01/07/echo-game/
http://mashable.com/2013/01/07/entrepreneur-trends-2013/
http://mashable.com/2013/01/07/

http://mashable.com/2013/02/07/nes-lunchbox/
http://mashable.com/2013/02/07/new-assassins-creed/
http://mashable.com/2013/02/07/new-york-times-digital-subscribers-2/
http://mashable.com/2013/02/07/nfl-cheerleader-video-bullies/
http://mashable.com/2013/02/07/nyclu-stop-and-frisk-watch-app-iphone/
http://mashable.com/2013/02/07/ouya-yearly-update/
http://mashable.com/2013/02/07/pepsis-coke-parody-helped-it-get-more-super-bowl-views-than-coke/
http://mashable.com/2013/02/07/photographer-disappearing-people/
http://mashable.com/2013/02/07/raul-labrador-me-likey-broke-girls/
http://mashable.com/2013/02/07/redbox-instant-xbox-review/
http://mashable.com/2013/02/07/report-iphone-grabs-36-percent-market-share-in-december/
http://mashable.com/2013/02/07/richard-iii-3d-printed-face/
http://mashable.com/2013/02/07/russia-tech-entrepreneurs/
http://mashable.com/2013/02/07/seth-macfarlane-host-oscars-academy-awards-erection/
http://mashable.com/2013/02/07/soccer-fan-michelin-guides/
http://mashabl

http://mashable.com/2013/03/15/facebook-cispa/
http://mashable.com/2013/03/15/facebook-teens/
http://mashable.com/2013/03/15/first-internet-domain-name/
http://mashable.com/2013/03/15/funny-vines/
http://mashable.com/2013/03/15/galaxy-s-4-hands-on/
http://mashable.com/2013/03/15/galaxy-s-4-iaccessories/
http://mashable.com/2013/03/15/galaxy-s-4-iphone-killer/
http://mashable.com/2013/03/15/google-reader-rss-void/
http://mashable.com/2013/03/15/google-spring-cleaning/
http://mashable.com/2013/03/15/grover-norquist-i-will-become-a-grumpy-cat-follower/
http://mashable.com/2013/03/15/hulu-acting-ceo-andy-forssell/
http://mashable.com/2013/03/15/jeopardy-taylor-swift/
http://mashable.com/2013/03/15/justice-is-served-jerk-bully-knocked-out-at-sxsw/
http://mashable.com/2013/03/15/kristen-stewart-history-st-patricks/
http://mashable.com/2013/03/15/lil-wayne-life-support-rumors-twitter/
http://mashable.com/2013/03/15/lucky-charms-auto-tune/
http://mashable.com/2013/03/15/mad-men-march-madness-s

http://mashable.com/2013/04/24/nbc-million-second-quiz/
http://mashable.com/2013/04/24/new-100-bill/
http://mashable.com/2013/04/24/next-generation-xbox-may-21/
http://mashable.com/2013/04/24/nintendo-weak-wii-u-sales/
http://mashable.com/2013/04/24/nokia-whatsapp/
http://mashable.com/2013/04/24/oblivion-movie-science/
http://mashable.com/2013/04/24/oculus-rift-comic/
http://mashable.com/2013/04/24/pakistan-tech-entrepreneurs/
http://mashable.com/2013/04/24/pixtr/
http://mashable.com/2013/04/24/reddit-boston-bombing/
http://mashable.com/2013/04/24/robin-van-persie-rvp-twitter-soccer-indian-it/
http://mashable.com/2013/04/24/samsung-galaxy-s-4-delayed-on-t-mobile/
http://mashable.com/2013/04/24/sony-digital-binoculars-2/
http://mashable.com/2013/04/24/strong-twitter-password/
http://mashable.com/2013/04/24/sugar-bush-squirrel/
http://mashable.com/2013/04/24/swimsuit/
http://mashable.com/2013/04/24/telepathy/
http://mashable.com/2013/04/24/the-bling-ring-trailer/
http://mashable.com/2013

http://mashable.com/2013/06/04/nettraveler-malware-cyberespionage-campaign/
http://mashable.com/2013/06/04/nfl-rodgers-game-of-thrones/
http://mashable.com/2013/06/04/nvidia-directstylus-tegra-4/
http://mashable.com/2013/06/04/obama-patent-trolls-actions/
http://mashable.com/2013/06/04/omgpop-zynga-layoffs/
http://mashable.com/2013/06/04/oscar-mayer-say-it-with-bacon/
http://mashable.com/2013/06/04/people-befriend-at-work/
http://mashable.com/2013/06/04/pipe-app-facebook/
http://mashable.com/2013/06/04/red-wedding-twitter/
http://mashable.com/2013/06/04/republican-chief-technology-officer-facebook/
http://mashable.com/2013/06/04/richard-saul-wurman/
http://mashable.com/2013/06/04/rickroll-breaks-vine/
http://mashable.com/2013/06/04/salesforce-exacttarget-acquisition/
http://mashable.com/2013/06/04/sean-parker-wedding-big-sur/
http://mashable.com/2013/06/04/social-media-advice-column-6-4/
http://mashable.com/2013/06/04/social-media-you/
http://mashable.com/2013/06/04/sony-announces-4-in

...

http://mashable.com/2014/08/14/netflix-confessions-whisper/
http://mashable.com/2014/08/14/new-york-times-audience-development/
http://mashable.com/2014/08/14/north-korea-fires-projectiles-pope/
http://mashable.com/2014/08/14/parrot-minidrones-review/
http://mashable.com/2014/08/14/pilot-prosthetic-arm/
http://mashable.com/2014/08/14/pizza-weird-things-vending-machines/
http://mashable.com/2014/08/14/pot-business-accounts/
http://mashable.com/2014/08/14/robin-williams-merry-friggin-christmas-clip-bourbon/
http://mashable.com/2014/08/14/robin-williams-was-in-early-stages-of-parkinsons-disease-his-wife-says/
http://mashable.com/2014/08/14/russias-prime-minister-twitter-account-gets-hacked/
http://mashable.com/2014/08/14/self-driving-car-accident-liability/
http://mashable.com/2014/08/14/sesame-street-uglydolls-3d-printed/
http://mashable.com/2014/08/14/shark-sex-facts/
http://mashable.com/2014/08/14/smart-tattoo-sweat/
http://mashable.com/2014/08/14/social-good-summit-michael-dell-robert

http://mashable.com/2014/09/16/dancing-traffic-light/
http://mashable.com/2014/09/16/dancing-with-the-stars-season-premiere-recap/
http://mashable.com/2014/09/16/dear-scotland-indyref/
http://mashable.com/2014/09/16/facebook-moments-private-sharing-apps/
http://mashable.com/2014/09/16/facebook-name-restrictions/
http://mashable.com/2014/09/16/first-3d-printed-car/
http://mashable.com/2014/09/16/flute-butterfly/
http://mashable.com/2014/09/16/giancarlo-stanton-face-pitch-photo/
http://mashable.com/2014/09/16/global-warming-fate-15-years/
http://mashable.com/2014/09/16/google-maps-gallery/
http://mashable.com/2014/09/16/i-know-what-you-did-last-summer-reboot/
http://mashable.com/2014/09/16/iphone-6-nfc-apple-pay/
http://mashable.com/2014/09/16/isis-slave-iraq/
http://mashable.com/2014/09/16/jacqui-lambie-moments/
http://mashable.com/2014/09/16/james-simmons-photographer-award/
http://mashable.com/2014/09/16/jimmy-fallon-barbra-streisand/
http://mashable.com/2014/09/16/kristen-wiig-and-el

http://mashable.com/2014/10/16/bbc-whatsapp-service-misinformation-ebola-west-africa/
http://mashable.com/2014/10/16/bermuda-hurricane-gonzalo/
http://mashable.com/2014/10/16/biden-selfies/
http://mashable.com/2014/10/16/bobs-burgers-business-tips/
http://mashable.com/2014/10/16/brad-pitt-fallon-breakdancing/
http://mashable.com/2014/10/16/california-shakes-it-off/
http://mashable.com/2014/10/16/cbs-digital-subscription-service/
http://mashable.com/2014/10/16/chinese-panda-sydney-psa/
http://mashable.com/2014/10/16/criminal-fiona-apple-cover-song-american-horror-story-freak-show-bette-dot/
http://mashable.com/2014/10/16/disney-selfies/
http://mashable.com/2014/10/16/dr-ruth-apple-ipad-event-tweets/
http://mashable.com/2014/10/16/drunk-j-crew-tumblr/
http://mashable.com/2014/10/16/ebola-blood-transfusions/
http://mashable.com/2014/10/16/ebola-cleveland-ohio/
http://mashable.com/2014/10/16/ebola-congress-hearing-nurse-texas/
http://mashable.com/2014/10/16/ebola-panic-fueling-rise-in-alar

http://mashable.com/2014/11/13/car-dance-taylor-swift/
http://mashable.com/2014/11/13/chinese-hackers-australian-sites/
http://mashable.com/2014/11/13/coffee-cup-comics/
http://mashable.com/2014/11/13/coldest-temperatures-united-states/
http://mashable.com/2014/11/13/comet-probe-space/
http://mashable.com/2014/11/13/compare-debt-g20-nations/
http://mashable.com/2014/11/13/directv-4k-samsung/
http://mashable.com/2014/11/13/ebola-devil-hole-sierra-leone/
http://mashable.com/2014/11/13/emotional-toddler/
http://mashable.com/2014/11/13/esa-scientist-sexist-shirt/
http://mashable.com/2014/11/13/eyelock-myris-review/
http://mashable.com/2014/11/13/facebook-privacy-basics/
http://mashable.com/2014/11/13/fifa-report-australia/
http://mashable.com/2014/11/13/fifa-report-qatar/
http://mashable.com/2014/11/13/global-warming-lightning/
http://mashable.com/2014/11/13/google-cardboard-volvo/
http://mashable.com/2014/11/13/hachette-detente-amazon/
http://mashable.com/2014/11/13/harry-potter-art/
http

http://mashable.com/2014/12/13/diy-meme-ornaments-2014/
http://mashable.com/2014/12/13/diy-trends-2014/
http://mashable.com/2014/12/13/dr-dreidel/
http://mashable.com/2014/12/13/facebook-drops-bing/
http://mashable.com/2014/12/13/gift-guide-fashionista/
http://mashable.com/2014/12/13/holiday-pinterest-fails/
http://mashable.com/2014/12/13/live-protests-dc-newyork/
http://mashable.com/2014/12/13/movie-theater-behavior/
http://mashable.com/2014/12/13/new-ferguson-documents/
http://mashable.com/2014/12/13/new-york-santacon-photos/
http://mashable.com/2014/12/13/nyc-dc-march-police-violence/
http://mashable.com/2014/12/13/parks-and-rec-goodbye/
http://mashable.com/2014/12/13/protestors-nationwide-police-brutality/
http://mashable.com/2014/12/13/santacon-nyc-protest/
http://mashable.com/2014/12/13/senate-spending-bill/
http://mashable.com/2014/12/13/sierra-leone-cancels-christmas-ebola/
http://mashable.com/2014/12/13/smaug-colbert-report/
http://mashable.com/2014/12/13/subscription-boxes-gi

In [14]:
# url

In [15]:
for each in range(0,len(url)):
    scrape(url[each])

{'title': 'Amazon', 'description': 'Amazon, the working name of Amazon.com Inc., is an online retailer and cloud computing company founded by Jeff Bezos in 1994. Based in Seattle, Amazon.com is one of the largest internet retailers in the world. The website began as an online bookstore, and later began selling video downloads and streaming, MP3 downloads and streaming, audiobooks, software, video games, electronics, apparel, furniture, food, toys and jewelry. The company now produces its own line of consumer electronics including the Amazon Kindle, Amazon Fire Tablet, Amazon Fire TV, and Amazon Echo products. It is also the world’s largest provider of cloud services. In August 2017, Amazon acquired supermarket chain Whole Foods Market for $13.4 billion. In September 2017, it announced that it would build a second headquarters to compliment its Seattle campus, and is still reviewing applications from cities across the country that have applied. The company employs over 300,000 people wo

MissingSchema: Invalid URL 'None': No schema supplied. Perhaps you meant http://None?